# first exploring the data

In [1]:
# Import packages
#import plotly.express as px
import pandas as pd
#from statsmodels.tsa.stattools import adfuller
import numpy as np
import sklearn as sk
## import pmdarima
import tensorflow
## import fprophet
## from pmdarima.arima import auto_arima
## from fbprophet import Prophet
## from tensorflow.keras.models import Sequential
## from tensorflow.keras.layers import LSTM, Dense
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


C:\Users\alaaw\AppData\Local\Temp\ipykernel_34364\102281708.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [85]:
# Read in the data
df = pd.read_excel("Sample Data Set (1).xlsx")
df.head()

,Reporting Economy,Product/Sector,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,World,SI3_AGG - TO - Total merchandise,10510291.0,12131279.0,14031816.0,16168718.0,12564776.0,15301659.0,18337603.0,18508981.0,18958769.0,18999593.0,16554643.0,16036335.0,17741325.0,19550037.0,19017108.0,17653275.0,22365766.0,24925766.0
1,World,SI3_AGG - AG - Agricultural products,847715.0,942450.0,1131691.0,1342823.0,1177136.0,1354429.0,1649389.0,1655099.0,1731531.0,1746687.0,1565686.0,1585693.0,1731381.0,1810271.0,1788200.0,1809475.0,2148665.0,2325521.0
2,World,SI3_AGG - AGFO - Food,680936.0,753836.0,915653.0,1114698.0,998004.0,1115790.0,1350316.0,1369778.0,1445339.0,1480252.0,1332886.0,1357202.0,1473948.0,1533561.0,1533159.0,1573538.0,1836335.0,2007666.0
3,World,SI3_AGG - MI - Fuels and mining products,1810185.0,2319402.0,2674500.0,3554158.0,2299789.0,3074398.0,4122493.0,4164125.0,4051300.0,3720251.0,2380017.0,2033684.0,2611396.0,3264056.0,3102597.0,2326889.0,3670710.0,4950975.0
4,World,SI3_AGG - MIFU - Fuels,1457315.0,1801129.0,2047440.0,2884170.0,1838528.0,2395065.0,3268505.0,3399965.0,3309479.0,3026072.0,1808811.0,1492255.0,1948478.0,2528513.0,2370333.0,1571511.0,2555970.0,3865877.0


checking the number of mising values in the data for each each column in our dataset

In [86]:
print(df.isnull().sum())

Reporting Economy      0
Product/Sector         0
2005                 415
2006                 409
2007                 436
2008                 402
2009                 377
2010                 355
2011                 332
2012                 292
2013                 320
2014                 286
2015                 273
2016                 272
2017                 239
2018                 221
2019                 191
2020                 166
2021                 171
2022                 219
dtype: int64


Exploring futher, to find out wither some rows should be deleted or not

In [95]:
missing_values_per_row = df.isnull().sum(axis=1)
rows_with_missing_values = missing_values_per_row[missing_values_per_row > 0]
sorted_indices = missing_values_per_row.sort_values(ascending=False).index
top_10_indices = sorted_indices[:10]
missing_values_df = missing_values_per_row[top_10_indices].reset_index()
missing_values_df.columns = ['index', 'missing_values']
# Merge the original DataFrame with the missing values count
result_df = df.loc[top_10_indices].reset_index().merge(missing_values_df, on='index')

print(result_df)

   index                 Reporting Economy  \
0   3398                        Uzbekistan   
1   1683                          Kiribati   
2    872  Democratic Republic of the Congo   
3   1016                          Eswatini   
4     88                            Angola   
5   2858                      Sierra Leone   
6    114               Antigua and Barbuda   
7   2219                             Nepal   
8   3488                             Yemen   
9   1206                        The Gambia   

                                      Product/Sector  2005  2006  2007  2008  \
0                             SI3_AGG - MIFU - Fuels   NaN   NaN   NaN   NaN   
1             SI3_AGG - MAMTTE - Transport equipment   NaN   NaN   NaN   NaN   
2                          SI3_AGG - MATE - Textiles   NaN   NaN   NaN   NaN   
3  SI3_AGG - MAMTOTEP - Electronic data processin...   2.0   1.0   NaN   NaN   
4    SI3_AGG - MAMTOF - Office and telecom equipment   NaN   NaN   NaN   NaN   
5            

In [100]:
rows_with_over_15_missing_values = missing_values_per_row[missing_values_per_row >= 1]

# Count the number of such rows
num_rows_with_over_15_missing = rows_with_over_15_missing_values.shape[0]

# Step 6: Calculate the percentage of these rows over the overall data
total_rows = df.shape[0]
percentage = (num_rows_with_over_15_missing / total_rows) * 100

# Output the results
print(f"Number of rows with more than 15 missing values: {num_rows_with_over_15_missing}")
print(f"Percentage of rows with more than 15 missing values: {percentage:.2f}%")

Number of rows with more than 15 missing values: 510
Percentage of rows with more than 15 missing values: 17.57%


In [90]:
##replacing 0 values with NaN for knn imputer 
df.replace(0, np.nan, inplace=True)

In [94]:
##drop rows with more then 10 NaN
MAXIMUM_NULLS = 10
df = df.dropna(thresh=len(df.columns) - MAXIMUM_NULLS)

In [99]:
# Count the rows where any column contains either 0 or NaN values
rows_with_zero_or_nan = (df == 0).any(axis=1) | df.isna().any(axis=1)
rows_with_zero_or_nan_count = rows_with_zero_or_nan.sum()

# Display the count of rows with any 0 or NaN values
rows_with_zero_or_nan_count

510

it seems like deleting the rows with missing values higer then 15, would mean sacrificing 1.87%. so we will explore some other options before choosing this option

Tuvalu seems to have a lot of missing data as well so i'll explore that too

In [103]:
### Select only numeric columns for imputation
numeric_data = df.select_dtypes(include=['number'])


In [104]:
print(numeric_data)

            2005        2006        2007        2008        2009        2010  \
0     10510291.0  12131279.0  14031816.0  16168718.0  12564776.0  15301659.0   
1       847715.0    942450.0   1131691.0   1342823.0   1177136.0   1354429.0   
2       680936.0    753836.0    915653.0   1114698.0    998004.0   1115790.0   
3      1810185.0   2319402.0   2674500.0   3554158.0   2299789.0   3074398.0   
4      1457315.0   1801129.0   2047440.0   2884170.0   1838528.0   2395065.0   
...          ...         ...         ...         ...         ...         ...   
3519         1.0         1.0         7.0         NaN         5.0         NaN   
3521         8.0       165.0        84.0        90.0        16.0        17.0   
3522         6.0        16.0        28.0        77.0        10.0         8.0   
3523        14.0        23.0        67.0        74.0        17.0        18.0   
3524         7.0        17.0        18.0        47.0        16.0         7.0   

            2011        2012        201

since the data depends on the region(rows) as well as the time i'll use KNNimputer which uses a proximaty(distance) metric to find the missing value. choose the n neighbors useing a hyper pramater that should be tuned

In [105]:
imputer = KNNImputer(n_neighbors=5, weights= 'distance')
##thinking of using the uneven weights of this method

In [106]:
## data preprocessing
imputed_data = imputer.fit_transform(numeric_data)
imputed_df = pd.DataFrame(imputed_data, columns=numeric_data.columns)
print(imputed_df.head())

         2005        2006        2007        2008        2009        2010  \
0  10510291.0  12131279.0  14031816.0  16168718.0  12564776.0  15301659.0   
1    847715.0    942450.0   1131691.0   1342823.0   1177136.0   1354429.0   
2    680936.0    753836.0    915653.0   1114698.0    998004.0   1115790.0   
3   1810185.0   2319402.0   2674500.0   3554158.0   2299789.0   3074398.0   
4   1457315.0   1801129.0   2047440.0   2884170.0   1838528.0   2395065.0   

         2011        2012        2013        2014        2015        2016  \
0  18337603.0  18508981.0  18958769.0  18999593.0  16554643.0  16036335.0   
1   1649389.0   1655099.0   1731531.0   1746687.0   1565686.0   1585693.0   
2   1350316.0   1369778.0   1445339.0   1480252.0   1332886.0   1357202.0   
3   4122493.0   4164125.0   4051300.0   3720251.0   2380017.0   2033684.0   
4   3268505.0   3399965.0   3309479.0   3026072.0   1808811.0   1492255.0   

         2017        2018        2019        2020        2021        2022 

In [107]:
X_data=df.iloc[:,:2]
X_data.head()

,Reporting Economy,Product/Sector
0,World,SI3_AGG - TO - Total merchandise
1,World,SI3_AGG - AG - Agricultural products
2,World,SI3_AGG - AGFO - Food
3,World,SI3_AGG - MI - Fuels and mining products
4,World,SI3_AGG - MIFU - Fuels


In [115]:
X_data_reset = X_data.reset_index(drop=True)
imputed_df_reset = imputed_df.reset_index(drop=True)
final_imputed_df = pd.concat([X_data_reset, imputed_df_reset], axis=1)
final_imputed_df.head()

,Reporting Economy,Product/Sector,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,World,SI3_AGG - TO - Total merchandise,10510291.0,12131279.0,14031816.0,16168718.0,12564776.0,15301659.0,18337603.0,18508981.0,18958769.0,18999593.0,16554643.0,16036335.0,17741325.0,19550037.0,19017108.0,17653275.0,22365766.0,24925766.0
1,World,SI3_AGG - AG - Agricultural products,847715.0,942450.0,1131691.0,1342823.0,1177136.0,1354429.0,1649389.0,1655099.0,1731531.0,1746687.0,1565686.0,1585693.0,1731381.0,1810271.0,1788200.0,1809475.0,2148665.0,2325521.0
2,World,SI3_AGG - AGFO - Food,680936.0,753836.0,915653.0,1114698.0,998004.0,1115790.0,1350316.0,1369778.0,1445339.0,1480252.0,1332886.0,1357202.0,1473948.0,1533561.0,1533159.0,1573538.0,1836335.0,2007666.0
3,World,SI3_AGG - MI - Fuels and mining products,1810185.0,2319402.0,2674500.0,3554158.0,2299789.0,3074398.0,4122493.0,4164125.0,4051300.0,3720251.0,2380017.0,2033684.0,2611396.0,3264056.0,3102597.0,2326889.0,3670710.0,4950975.0
4,World,SI3_AGG - MIFU - Fuels,1457315.0,1801129.0,2047440.0,2884170.0,1838528.0,2395065.0,3268505.0,3399965.0,3309479.0,3026072.0,1808811.0,1492255.0,1948478.0,2528513.0,2370333.0,1571511.0,2555970.0,3865877.0


In [116]:
print(final_imputed_df)

     Reporting Economy                                     Product/Sector  \
0                World                   SI3_AGG - TO - Total merchandise   
1                World               SI3_AGG - AG - Agricultural products   
2                World                              SI3_AGG - AGFO - Food   
3                World           SI3_AGG - MI - Fuels and mining products   
4                World                             SI3_AGG - MIFU - Fuels   
...                ...                                                ...   
2897          Zimbabwe  SI3_AGG - MAMTOTTL - Telecommunications equipment   
2898          Zimbabwe             SI3_AGG - MAMTTE - Transport equipment   
2899          Zimbabwe             SI3_AGG - MAMTAU - Automotive products   
2900          Zimbabwe                          SI3_AGG - MATE - Textiles   
2901          Zimbabwe                          SI3_AGG - MACL - Clothing   

            2005        2006        2007        2008        2009        201

In [126]:
df_long = final_imputed_df.copy()

In [127]:
df_long = pd.melt(df_long, id_vars=['Product/Sector','Reporting Economy'], value_vars=[ '2005',
'2006','2007','2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'
], var_name='Year', value_name='Value')

In [130]:
df_long.to_excel('NanOver9removedlong.xlsx', index=False)

In [117]:
##output the data 
final_imputed_df.to_excel('NanOver9removed.xlsx', index=False)